# Convolutional Neural Network

### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
# the dataset isn't numbers this time so no pandas or numpy

2023-10-10 20:53:42.625049: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 20:54:21.568059: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 20:54:21.568095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 20:54:21.791736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 20:54:46.191645: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 20:54:46.347964: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
tf.__version__

'2.14.0'

## Part 1- Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# these random transformations help prevent overfitting 
# (the CNN finding features that are common to the pictures in general but not the target)
training_set = train_datagen.flow_from_directory(
    "dataset/training_set/",
    target_size=(64, 64),
    batch_size=32, 
    class_mode="binary"
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    "dataset/test_set/",
    target_size=(64, 64),
    batch_size=32,
    class_mode="binary"
)

Found 2000 images belonging to 2 classes.


## Part 2- Building the CNN

### Initialising the CNN

In [15]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=[64, 64, 3]))
# creates the feature map

### Step 2 - Pooling

In [17]:
cnn.add((tf.keras.layers.MaxPool2D(pool_size=2, strides=2)))
# transforms the feature map into a pooled feature map
# a 2x2 frame and then how many pixels it shifts each move

### Adding a second convolutional layer

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
cnn.add((tf.keras.layers.MaxPool2D(pool_size=2, strides=2)))
# pools it down one more time
# doesn't need an input shape because it's not connected to the input layer

### Step 3 - Flattening

In [19]:
cnn.add(tf.keras.layers.Flatten())
# turns the matrix into a 1d vector

### Step 4 - Full Connection

In [20]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu", input_shape = (None, 100)))
# Needs way more neurons than number predictions

### Step 5 - Output Layer

In [21]:
cnn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Part 3 - Training the CNN

### Compiling the CNN

In [22]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# It's also binary classification so the same as the ANN

### Training the CNN on the Training set and evaluating it on the Test set

In [26]:
cnn.fit(x= training_set, validation_data=test_set, epochs=5)
# The lecture recommended 25 epochs but that would run through my Codespace resources too fast so I set it to 5

Epoch 1/5
250/250 [==============================] - 16s 62ms/step - loss: 0.6011 - accuracy: 0.6827 - val_loss: 0.5855 - val_accuracy: 0.6815
Epoch 2/5
250/250 [==============================] - 16s 62ms/step - loss: 0.5484 - accuracy: 0.7214 - val_loss: 0.5628 - val_accuracy: 0.7140
Epoch 3/5
250/250 [==============================] - 16s 62ms/step - loss: 0.5254 - accuracy: 0.7390 - val_loss: 0.5166 - val_accuracy: 0.7460
Epoch 4/5
250/250 [==============================] - 15s 61ms/step - loss: 0.4974 - accuracy: 0.7551 - val_loss: 0.4977 - val_accuracy: 0.7650
Epoch 5/5
250/250 [==============================] - 15s 62ms/step - loss: 0.4842 - accuracy: 0.7670 - val_loss: 0.5118 - val_accuracy: 0.7575


## Part 4 - Making a single prediction

In [37]:
import numpy as np
# oops, needed numpy after all!
from keras.preprocessing import image
test_image = image.load_img("dataset/single_prediction/cat_or_dog_4.jpg", target_size=(64, 64))
# test images need to be the same size as the target size.
test_image = image.img_to_array(test_image)
# converts the image into an array of pixel data
test_image = np.expand_dims(test_image, axis=0)
# Needs to be in a batch, like the training images
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0]  == 0: # the image is now inside of an array two dimensions deep, but is index 0 in both dimensions
    prediction = "cat"
else:
    prediction = "dog"


1/1 [==============================] - 0s 13ms/step


In [38]:
print(prediction)

dog
